In [2]:
import imdb as db
from sqlalchemy import select
from sqlalchemy.orm import Session

In [ ]:
stm = (
    select(db.Actor)
    .join(db.Actor.roles)
    .join(db.Role.movie)
    .join(db.Movie.genres)
    .where(db.MoviesGenres.genre == 'War')
)

In [ ]:
s = Session(db.engine)
res = s.execute(stm)
for i in res.scalars():
  print(i.last_name)



In [7]:
stm = (select(
    db.Actor.first_name,
    db.Actor.last_name,
    db.Actor.gender,
    db.Role.role,
    db.Movie.name,
    db.Movie.year,
    db.MoviesGenres.genre,
    db.Director.first_name,
    db.Director.last_name
    )
    .join(db.Role, db.Role.actor_id == db.Actor.id)
    .join(db.Movie, db.Role.movie_id == db.Movie.id)
    .join(db.MoviesDirectors, db.MoviesDirectors.movie_id == db.Movie.id)
    .join(db.Director, db.MoviesDirectors.director_id == db.Director.id)
    .join(db.MoviesGenres, db.Movie.id == db.MoviesGenres.movie_id)
    .order_by(db.Movie.id)
)


In [8]:
f = open('small_data.csv','wt')
print('actor_first_name,actor_last_name,actor_gender,actor_role,movie_name,movie_year,movie_genre,direcor_first_name,director_last_name',file=f)

s = Session(db.engine)
res = s.execute(stm)
for r in res:
    print(*r,sep=',',file=f)
s.commit()
f.close()


2022-10-03 01:58:50,964 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 01:58:50,976 INFO sqlalchemy.engine.Engine SELECT actors.first_name, actors.last_name, actors.gender, roles.role, movies.name, movies.year, movies_genres.genre, directors.first_name AS first_name_1, directors.last_name AS last_name_1 
FROM actors JOIN roles ON roles.actor_id = actors.id JOIN movies ON roles.movie_id = movies.id JOIN movies_directors ON movies_directors.movie_id = movies.id JOIN directors ON movies_directors.director_id = directors.id JOIN movies_genres ON movies.id = movies_genres.movie_id ORDER BY movies.id
2022-10-03 01:58:50,978 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ()
2022-10-03 01:58:51,232 INFO sqlalchemy.engine.Engine COMMIT


~~~sql
select
  actors.*
from
  actors,
  roles
where
  actors.id = roles.actor_id
  and roles.movie_id not in (
    select
      movie_id
    from
      movies_genres
    where
      movies_genres.genre = 'War'
  )
~~~